In [1]:
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
from datetime import datetime
import json
import os
import requests
from MySQLdb import _mysql

In [2]:
# INPUTS
# ---------------------------------------------------------------------------------------------
etf_landing_page = 'https://www.ishares.com/us/products/239707/ishares-russell-1000-etf'
# start_date = '2017-12-31'
start_date = '2023-01-01'
end_date = datetime.now().strftime('%Y-%m-%d')
output_file = r'C:\Users\Jerry\Downloads\Russell-1000-Holdings.csv'

In [3]:
def get_trading_day_month_ends(exchange_code, start_date, end_date, output_format):
    nyse = mcal.get_calendar(exchange_code)
    trading_days_df = pd.DataFrame({'trading_date':
                                        nyse.valid_days(start_date=start_date,
                                                        end_date=end_date)
                                    })
    trading_days_df['yymm'] = trading_days_df['trading_date'].dt.strftime('%y%m')
    trading_day_month_ends = trading_days_df.groupby('yymm')['trading_date'].max(). \
        dt.strftime(output_format).to_list()

    return trading_day_month_ends

In [4]:
def map_raw_item(unmapped_item):
    return {
        'ticker': unmapped_item[0],
        'name': unmapped_item[1],
        'sector': unmapped_item[2],
        'asset_class': unmapped_item[3],
        'market_value': unmapped_item[4]['raw'],
        'weight': unmapped_item[5]['raw'],
        'notional_value': unmapped_item[6]['raw'],
        'shares': unmapped_item[7]['raw'],
        'cusip': unmapped_item[8],
        'isin': unmapped_item[9],
        'sedol': unmapped_item[10],
        'price': unmapped_item[11]['raw'],
        'location': unmapped_item[12],
        'exchange': unmapped_item[13],
        'currency': unmapped_item[14],
        'fx_rate': unmapped_item[15],
        'maturity': unmapped_item[16]
    }

In [5]:
def format_response(response_json):
    input_items = response_json['aaData']
    output_items = []
    for input_item in input_items:
        mapped_item = map_raw_item(input_item)
        output_items.append(mapped_item)
    return (output_items)

In [6]:
list_of_holdings = []
yyyymmdd_queue = get_trading_day_month_ends('NYSE', start_date, end_date, '%Y%m%d')

In [7]:
for yyyymmdd in yyyymmdd_queue:
    request_url = f'{etf_landing_page}/1467271812596.ajax?' \
                  f'fileType=json&tab=all&asOfDate={yyyymmdd}'
    print(f'requesting: {request_url}')

    response = requests.get(request_url)

    if response is None:
        print(f'{"!" * 10}\nREQUEST FAILED({yyyymmdd}):\n'
              f'ERROR: Request to SB Timed-Out\n'
              f'{"!" * 10}')
    else:
        print(f'response.status_code: {response.status_code}')

        if response.status_code != 200:

            print(f'{"!" * 10}\nREQUEST FAILED({yyyymmdd}):\n'
                  f'status code = {response.status_code}\n'
                  f'response_message = {response.text}\n'
                  f'{"!" * 10}')
        else:
            response_json = json.loads(response.content)
            holdings_json = format_response(response_json)
            holdings_df = pd.DataFrame(holdings_json)

            if holdings_df.shape[0] == 0:
                print(f'{"!" * 10}\nERROR: 0 ROWS ({yyyymmdd})\n{"!" * 10}')

            else:
                print(f'number of rows: {holdings_df.shape[0]}')

                # add date col
                holdings_df.insert(loc=0,
                                   column='as_of_date',
                                   value=f'{yyyymmdd[:4]}-{yyyymmdd[4:6]}-{yyyymmdd[6:]}')  # %Y-%m-%d
                list_of_holdings.append(holdings_df)

requesting: https://www.ishares.com/us/products/239707/ishares-russell-1000-etf/1467271812596.ajax?fileType=json&tab=all&asOfDate=20230131
response.status_code: 200
number of rows: 1017
requesting: https://www.ishares.com/us/products/239707/ishares-russell-1000-etf/1467271812596.ajax?fileType=json&tab=all&asOfDate=20230228
response.status_code: 200
number of rows: 1015
requesting: https://www.ishares.com/us/products/239707/ishares-russell-1000-etf/1467271812596.ajax?fileType=json&tab=all&asOfDate=20230331
response.status_code: 200
number of rows: 1012
requesting: https://www.ishares.com/us/products/239707/ishares-russell-1000-etf/1467271812596.ajax?fileType=json&tab=all&asOfDate=20230428
response.status_code: 200
number of rows: 1013
requesting: https://www.ishares.com/us/products/239707/ishares-russell-1000-etf/1467271812596.ajax?fileType=json&tab=all&asOfDate=20230531
response.status_code: 200
number of rows: 1012
requesting: https://www.ishares.com/us/products/239707/ishares-russell

In [8]:
holdings = pd.concat(list_of_holdings)

In [9]:
clean_holdings = holdings[(holdings['asset_class'] == "Equity") & (holdings['ticker'] != "-") & (holdings['cusip'] != "-")]

In [10]:
# Group by 'ticker' and aggregate using first non-NA value
security_data = clean_holdings.groupby('cusip').agg({
    'ticker': 'last',
    'name': 'last',
    'sector': 'last',
    'asset_class': 'last',
    'location': 'last',
    'exchange': 'last'
}).reset_index()

In [11]:
security_data

,cusip,ticker,name,sector,asset_class,location,exchange
0,00090Q103,ADT,ADT INC,Consumer Discretionary,Equity,United States,New York Stock Exchange Inc.
1,001055102,AFL,AFLAC INC,Financials,Equity,United States,New York Stock Exchange Inc.
2,001084102,AGCO,AGCO CORP,Industrials,Equity,United States,New York Stock Exchange Inc.
3,00123Q104,AGNC,AGNC INVESTMENT REIT CORP,Financials,Equity,United States,NASDAQ
4,00130H105,AES,AES CORP,Utilities,Equity,United States,New York Stock Exchange Inc.
...,...,...,...,...,...,...,...
1008,G8473T100,STE,STERIS,Health Care,Equity,United States,New York Stock Exchange Inc.
1009,G9618E107,WTM,WHITE MOUNTAINS INSURANCE GROUP LT,Financials,Equity,United States,New York Stock Exchange Inc.
1010,N72482123,QGEN,QIAGEN NV,Health Care,Equity,Germany,New York Stock Exchange Inc.
1011,P31076105,CPA,COPA HOLDINGS A SA,Industrials,Equity,Panama,New York Stock Exchange Inc.


In [12]:
# Connecting to the MySQL server
db = _mysql.connect(host="localhost", user="root", password="password")

try:
    # Select the database
    db.query("USE edgar_db_test")
except _mysql.OperationalError:
    db.query("CREATE DATABASE edgar_db_test")
    db.query("USE edgar_db_test")
    
# Query to create the security_info table
create_table_query = """
CREATE TABLE IF NOT EXISTS security_info (
    cusip CHAR(9) PRIMARY KEY NOT NULL,
    ticker VARCHAR(255),
    name VARCHAR(255),
    sector VARCHAR(255),
    asset_class VARCHAR(255),
    location VARCHAR(255),
    exchange VARCHAR(255)
);
"""

db.query(create_table_query)

In [13]:
# Iterate through each row in the DataFrame
for index, row in security_data.iterrows():
    cusip_value = row['cusip']
    
    # Check if the record already exists in the table
    result = db.query(f"SELECT 1 FROM security_info WHERE cusip = '{cusip_value}'")
    
    if db.use_result().num_rows():
        print(cusip_value, "already exists")
    else:
        # If not, insert the row to the table
        try:
            insert_query = f"""
            INSERT INTO security_info (
                cusip,
                ticker,
                name,
                sector,
                asset_class,
                location,
                exchange
            ) 
            VALUES (
                '{row['cusip']}',
                '{row['ticker']}',
                '{row['name']}',
                '{row['sector']}',
                '{row['asset_class']}',
                '{row['location']}',
                '{row['exchange']}'
            )
            """
            db.query(insert_query)
        except _mysql.MySQLError as e:
            print("Insert failed", e)

In [28]:
db.query("SELECT 1 FROM security_info WHERE cusip = '26614N102'")
result = db.store_result().fetch_row()

In [29]:
result

((b'1',),)

In [24]:
db.query(f"SELECT 1 FROM security_info WHERE cusip = '26614N102' LIMIT 1")

In [25]:
result = db.store_result().fetch_row()

In [22]:
db.use_result().num_rows()

0

In [18]:
db.query(f"SELECT 1 FROM security_info WHERE cusip = '001W0310X' LIMIT 1")

In [19]:
db.use_result().num_rows()

0

In [26]:
result

((b'1',),)

In [55]:
db.query("""SELECT 1 FROM security_info WHERE cusip = '28176E108@'""")

In [56]:
result = db.store_result()

In [57]:
result.fetch_row()

()

In [58]:
result.num_rows()

0